In [1]:
import scantelligent.functions as st
import time
from time import sleep
from pynput import keyboard
import nanonispy as nap
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from datetime import datetime

Start a session

In [ ]:
session_path, images_path, logfile_path = st.initialize()

Simple auto approach

In [ ]:
st.auto_approach(I_approach = 100, p_gain_approach = 30, t_const_approach = 167, wait = True, land_tip = True, timeout = 1000, z_voltage = 220)

Simple coarse move in any direction

In [ ]:
st.move_over(direction = "north", steps = 20, z_steps = 10, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 220, land_tip = True)

In [ ]:
session_path, images_path, logfile_path = st.initialize()
st.change_bias(-3)
st.move_over(direction = "up", steps = 1, z_steps = 1)
st.tip_tracker(sampling_time = .5, velocity_threshold = .1, timeout = 60, N_no_motion = 4, exit_when_still = True)
sleep(5)
st.change_bias(-1.6)
st.scan_control("start", scan_direction = "down")
st.tip_tracker(sampling_time = .5, velocity_threshold = .1, timeout = 60, N_no_motion = 4, exit_when_still = True)
st.logprint("Scan completed!")

In [ ]:
st.move_over(direction = "east", steps = 500, z_steps = 20, z_voltage = 240)

In [ ]:
image = st.get_latest_scan(preferred_channels = ["Z", "LI Demod 1 X (A)"], scan_direction = "forward", subtraction = "plane", clip_fraction = .01)
plt.imshow(image, cmap = "gray")
plt.show()

In [3]:
session_path = st.get_session_path()
file_data = st.get_file_data(session_path = session_path)
sxm_files = file_data.image_files
scan_data = st.get_scan(sxm_files[3])
z_img = scan_data.scans[0, 0]

In [ ]:
clip_values = st.clip_range(z_img, method = "percentiles", values = [.2, .8])
histogram = st.get_image_statistics(z_img).histogram
z_img_b = st.background_subtract(z_img)
clip_values_b = st.clip_range(z_img_b, method = "percentiles", values = [.2, .8])
histogram_b = st.get_image_statistics(z_img_b).histogram

fig, ax = plt.subplots(1, 3)
ax[0].imshow(z_img, vmin = clip_values[0], vmax = clip_values[1], cmap = "gray")
ax[1].plot(-histogram[1], histogram[0] - np.mean(histogram[0]))
ax[1].plot(histogram_b[1], histogram_b[0])
ax[2].imshow(z_img_b, vmin = clip_values_b[0], vmax = clip_values_b[1], cmap = "gray")
fig.set_size_inches(10, 5)
plt.show()